# Music Recommendation System with Spotify Data
### 1. Load and preprocess data 


In [57]:
import pandas as pd

file_path = 'data/spotify_data.csv'
df = pd.read_csv(file_path)

In [58]:
print(df.head(5))
df.info()

   Unnamed: 0    artist_name        track_name                track_id  \
0           0     Jason Mraz   I Won't Give Up  53QF56cjZA9RTuuMZDrSA6   
1           1     Jason Mraz  93 Million Miles  1s8tP3jP4GZcyHDsjvw218   
2           2  Joshua Hyslop  Do Not Let Me Go  7BRCa8MPiyuvr2VU3O9W0F   
3           3   Boyce Avenue          Fast Car  63wsZUhUZLlh1OsyrZq7sz   
4           4   Andrew Belle  Sky's Still Blue  6nXIYClvJAfi6ujLiKqEq8   

   popularity  year     genre  danceability  energy  key  loudness  mode  \
0          68  2012  acoustic         0.483   0.303    4   -10.058     1   
1          50  2012  acoustic         0.572   0.454    3   -10.286     1   
2          57  2012  acoustic         0.409   0.234    3   -13.711     1   
3          58  2012  acoustic         0.392   0.251   10    -9.845     1   
4          54  2012  acoustic         0.430   0.791    6    -5.419     0   

   speechiness  acousticness  instrumentalness  liveness  valence    tempo  \
0       0.0429      

In [59]:
# View missing values
print(df.isnull().sum())

# Drop rows with missing values
df = df.dropna() 

Unnamed: 0           0
artist_name         15
track_name           1
track_id             0
popularity           0
year                 0
genre                0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
duration_ms          0
time_signature       0
dtype: int64


### Audio Features Overview

Below is a table summarizing key audio features of a track, along with a brief explanation of what each feature represents.

| **Audio Feature**   | **Description**                                | **What It Tells Us**                                                                 |
|---------------------|-----------------------------------------------|-------------------------------------------------------------------------------------|
| **Popularity**      | Track popularity (0 to 100)                   | How popular the track is, with higher values indicating greater popularity.         |
| **Year**            | Year released (2000 to 2023)                  | The release year of the track, useful for historical context or trends.             |
| **Danceability**    | Track suitability for dancing (0.0 to 1.0)    | How suitable the track is for dancing, based on rhythm, tempo, and beat strength.   |
| **Energy**          | Perceptual measure of intensity and activity (0.0 to 1.0) | The intensity and activity level of the track; higher values feel more energetic.   |
| **Key**             | The key the track is in (-1 to 11)            | The musical key of the track (e.g., C, D#, etc.), where -1 means no key detected.   |
| **Loudness**        | Overall loudness in decibels (-60 to 0 dB)    | How loud the track is; closer to 0 dB means louder, more impactful sound.           |
| **Mode**            | Modality of the track (Major = 1, Minor = 0)  | Whether the track is in a major (brighter) or minor (darker) key, affecting mood.   |
| **Speechiness**     | Presence of spoken words in the track         | How much spoken word is in the track; higher values indicate more speech-like content. |
| **Acousticness**    | Confidence measure from 0 to 1 of whether the track is acoustic | Likelihood of the track being acoustic; higher values mean more acoustic elements.  |
| **Instrumentalness**| Whether tracks contain vocals (0.0 to 1.0)    | Likelihood of the track being instrumental; higher values mean fewer vocals.         |
| **Liveness**        | Presence of audience in the recording (0.0 to 1.0) | Whether the track sounds like a live recording; higher values suggest an audience.  |
| **Valence**         | Musical positiveness (0.0 to 1.0)             | The emotional tone of the track; higher values indicate a more positive, happy mood. |
| **Tempo**           | Tempo of the track in beats per minute (BPM)  | The speed of the track; higher BPM means a faster tempo, often more energetic.       |
| **Time_signature**  | Estimated time signature (3 to 7)             | The rhythmic structure (e.g., 4/4, 3/4); indicates beats per measure.               |
| **Duration_ms**     | Duration of track in milliseconds             | The length of the track in milliseconds; useful for understanding track duration.    |

I chose to use as features : ***Danceability, Energy, Key, Loudness, Mode, Speechiness, Acousticness, Instrumentalness, Liveness, Valence, and Tempo***
- For **tempo**: Apply min-max scaling to bring its values into the [0, 1] interval. If tempo contains notable outliers, consider robust scaling.  
- For **categorical** features (key and mode): Use one-hot encoding rather than numeric scaling.

In [60]:
# Define the chosen features
features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
           'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

df = df[features].copy()

# Apply min-max scaling to tempo to bring it into [0, 1] interval
from sklearn.preprocessing import MinMaxScaler

# Check for tempo outliers
print("Tempo statistics:")
print(df['tempo'].describe())

# Apply min-max scaling to tempo
scaler = MinMaxScaler()
df['tempo'] = scaler.fit_transform(df[['tempo']])

# One-hot encode categorical features (key and mode)
# For key (values 0-11 representing musical keys)
key_dummies = pd.get_dummies(df['key'], prefix='key')
df = pd.concat([df, key_dummies], axis=1)

# For mode (0 for minor, 1 for major)
mode_dummies = pd.get_dummies(df['mode'], prefix='mode')
df = pd.concat([df, mode_dummies], axis=1)

# Drop the original categorical columns
df.drop(['key', 'mode'], axis=1, inplace=True)

# Display the first few rows of the processed features
print("\nProcessed features:")
print(df.head())

# Display the shape of the processed features dataframe
print(f"\nShape of processed features: {df.shape}")


Tempo statistics:
count    1.159748e+06
mean     1.213775e+02
std      2.977964e+01
min      0.000000e+00
25%      9.879800e+01
50%      1.219310e+02
75%      1.399030e+02
max      2.499930e+02
Name: tempo, dtype: float64

Processed features:
   danceability  energy  loudness  speechiness  acousticness  \
0         0.483   0.303   -10.058       0.0429        0.6940   
1         0.572   0.454   -10.286       0.0258        0.4770   
2         0.409   0.234   -13.711       0.0323        0.3380   
3         0.392   0.251    -9.845       0.0363        0.8070   
4         0.430   0.791    -5.419       0.0302        0.0726   

   instrumentalness  liveness  valence     tempo  key_0  ...  key_4  key_5  \
0          0.000000    0.1150    0.139  0.533639  False  ...   True  False   
1          0.000014    0.0974    0.515  0.560744  False  ...  False  False   
2          0.000050    0.0895    0.145  0.559344  False  ...  False  False   
3          0.000000    0.0797    0.508  0.819867  False  ...